In [1]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pprint
import pyspark
import pyspark.sql.functions as F

from pyspark.sql.functions import col
from pyspark.sql.types import StringType, IntegerType, FloatType, DateType

import utils.lms.lms_data_processing_bronze_table
import utils.clickstream.clickstream_data_processing_bronze_table
import utils.attributes.attributes_data_processing_bronze_table
import utils.financials.financials_data_processing_bronze_table

import utils.lms.lms_data_processing_silver_table
# import utils.clickstream.clickstream_data_processing_silver_table
# import utils.attributes.attributes_data_processing_silver_table
# import utils.financials.financials_data_processing_silver_table

# import utils.data_processing_gold_table
print('Done!')

Done!


## Set up PySpark session

In [2]:
# Initialize SparkSession
spark = pyspark.sql.SparkSession.builder \
    .appName("dev") \
    .master("local[*]") \
    .getOrCreate()

# Set log level to ERROR to hide warnings
spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/18 08:53:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Set up Config

In [3]:
# set up config
snapshot_date_str = "2023-01-01"

start_date_str = "2023-01-01"
end_date_str = "2025-12-01"

In [4]:
# generate list of dates to process
def generate_first_of_month_dates(start_date_str, end_date_str):
    # Convert the date strings to datetime objects
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
    end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
    
    # List to store the first of month dates
    first_of_month_dates = []

    # Start from the first of the month of the start_date
    current_date = datetime(start_date.year, start_date.month, 1)

    while current_date <= end_date:
        # Append the date in yyyy-mm-dd format
        first_of_month_dates.append(current_date.strftime("%Y-%m-%d"))
        
        # Move to the first of the next month
        if current_date.month == 12:
            current_date = datetime(current_date.year + 1, 1, 1)
        else:
            current_date = datetime(current_date.year, current_date.month + 1, 1)

    return first_of_month_dates

dates_str_lst = generate_first_of_month_dates(start_date_str, end_date_str)
dates_str_lst

['2023-01-01',
 '2023-02-01',
 '2023-03-01',
 '2023-04-01',
 '2023-05-01',
 '2023-06-01',
 '2023-07-01',
 '2023-08-01',
 '2023-09-01',
 '2023-10-01',
 '2023-11-01',
 '2023-12-01',
 '2024-01-01',
 '2024-02-01',
 '2024-03-01',
 '2024-04-01',
 '2024-05-01',
 '2024-06-01',
 '2024-07-01',
 '2024-08-01',
 '2024-09-01',
 '2024-10-01',
 '2024-11-01',
 '2024-12-01',
 '2025-01-01',
 '2025-02-01',
 '2025-03-01',
 '2025-04-01',
 '2025-05-01',
 '2025-06-01',
 '2025-07-01',
 '2025-08-01',
 '2025-09-01',
 '2025-10-01',
 '2025-11-01',
 '2025-12-01']

## Build Bronze Table

### LMS

In [5]:
# create bronze datalake
# create multiple csv files 
bronze_lms_directory = "datamart/bronze/lms/"

if not os.path.exists(bronze_lms_directory):
    os.makedirs(bronze_lms_directory)

In [6]:
# run bronze backfill
for date_str in dates_str_lst:
    utils.lms.lms_data_processing_bronze_table.process_bronze_table(date_str, bronze_lms_directory, spark)

2023-01-01row count: 530
saved to: datamart/bronze/lms/bronze_loan_daily_2023_01_01.csv
2023-02-01row count: 1031
saved to: datamart/bronze/lms/bronze_loan_daily_2023_02_01.csv
2023-03-01row count: 1537
saved to: datamart/bronze/lms/bronze_loan_daily_2023_03_01.csv
2023-04-01row count: 2047
saved to: datamart/bronze/lms/bronze_loan_daily_2023_04_01.csv
2023-05-01row count: 2568
saved to: datamart/bronze/lms/bronze_loan_daily_2023_05_01.csv
2023-06-01row count: 3085
saved to: datamart/bronze/lms/bronze_loan_daily_2023_06_01.csv
2023-07-01row count: 3556
saved to: datamart/bronze/lms/bronze_loan_daily_2023_07_01.csv
2023-08-01row count: 4037
saved to: datamart/bronze/lms/bronze_loan_daily_2023_08_01.csv
2023-09-01row count: 4491
saved to: datamart/bronze/lms/bronze_loan_daily_2023_09_01.csv
2023-10-01row count: 4978
saved to: datamart/bronze/lms/bronze_loan_daily_2023_10_01.csv
2023-11-01row count: 5469
saved to: datamart/bronze/lms/bronze_loan_daily_2023_11_01.csv
2023-12-01row count: 5

In [7]:
# inspect output
utils.lms.lms_data_processing_bronze_table.process_bronze_table(dates_str_lst[0], bronze_lms_directory, spark).toPandas()

2023-01-01row count: 530
saved to: datamart/bronze/lms/bronze_loan_daily_2023_01_01.csv


,loan_id,Customer_ID,loan_start_date,tenure,installment_num,loan_amt,due_amt,paid_amt,overdue_amt,balance,snapshot_date
0,CUS_0x1037_2023_01_01,CUS_0x1037,2023-01-01,10,0,10000,0.0,0.0,0.0,10000.0,2023-01-01
1,CUS_0x1069_2023_01_01,CUS_0x1069,2023-01-01,10,0,10000,0.0,0.0,0.0,10000.0,2023-01-01
2,CUS_0x114a_2023_01_01,CUS_0x114a,2023-01-01,10,0,10000,0.0,0.0,0.0,10000.0,2023-01-01
3,CUS_0x1184_2023_01_01,CUS_0x1184,2023-01-01,10,0,10000,0.0,0.0,0.0,10000.0,2023-01-01
4,CUS_0x1297_2023_01_01,CUS_0x1297,2023-01-01,10,0,10000,0.0,0.0,0.0,10000.0,2023-01-01
...,...,...,...,...,...,...,...,...,...,...,...
525,CUS_0xe98_2023_01_01,CUS_0xe98,2023-01-01,10,0,10000,0.0,0.0,0.0,10000.0,2023-01-01
526,CUS_0xea6_2023_01_01,CUS_0xea6,2023-01-01,10,0,10000,0.0,0.0,0.0,10000.0,2023-01-01
527,CUS_0xed3_2023_01_01,CUS_0xed3,2023-01-01,10,0,10000,0.0,0.0,0.0,10000.0,2023-01-01
528,CUS_0xed8_2023_01_01,CUS_0xed8,2023-01-01,10,0,10000,0.0,0.0,0.0,10000.0,2023-01-01


### Features Clickstream

In [8]:
# create bronze datalake
# create multiple csv files 
bronze_clickstream_directory = "datamart/bronze/clickstream/"

if not os.path.exists(bronze_clickstream_directory):
    os.makedirs(bronze_clickstream_directory)

In [9]:
# run bronze backfill
for date_str in dates_str_lst:
    utils.clickstream.clickstream_data_processing_bronze_table.process_bronze_table(date_str, bronze_clickstream_directory, spark)

2023-01-01row count: 8974
saved to: datamart/bronze/clickstream/bronze_clickstream_2023_01_01.csv
2023-02-01row count: 8974
saved to: datamart/bronze/clickstream/bronze_clickstream_2023_02_01.csv
2023-03-01row count: 8974
saved to: datamart/bronze/clickstream/bronze_clickstream_2023_03_01.csv
2023-04-01row count: 8974
saved to: datamart/bronze/clickstream/bronze_clickstream_2023_04_01.csv
2023-05-01row count: 8974
saved to: datamart/bronze/clickstream/bronze_clickstream_2023_05_01.csv
2023-06-01row count: 8974
saved to: datamart/bronze/clickstream/bronze_clickstream_2023_06_01.csv
2023-07-01row count: 8974
saved to: datamart/bronze/clickstream/bronze_clickstream_2023_07_01.csv
2023-08-01row count: 8974
saved to: datamart/bronze/clickstream/bronze_clickstream_2023_08_01.csv
2023-09-01row count: 8974
saved to: datamart/bronze/clickstream/bronze_clickstream_2023_09_01.csv
2023-10-01row count: 8974
saved to: datamart/bronze/clickstream/bronze_clickstream_2023_10_01.csv
2023-11-01row count:

In [10]:
# inspect output
utils.clickstream.clickstream_data_processing_bronze_table.process_bronze_table(dates_str_lst[0], bronze_clickstream_directory, spark).toPandas()

2023-01-01row count: 8974
saved to: datamart/bronze/clickstream/bronze_clickstream_2023_01_01.csv


,fe_1,fe_2,fe_3,fe_4,fe_5,fe_6,fe_7,fe_8,fe_9,fe_10,...,fe_13,fe_14,fe_15,fe_16,fe_17,fe_18,fe_19,fe_20,Customer_ID,snapshot_date
0,63,118,80,121,55,193,111,112,-101,83,...,-16,-81,-126,114,35,85,-73,76,CUS_0x1037,2023-01-01
1,-108,182,123,4,-56,27,25,-6,284,222,...,-14,-96,200,35,130,94,111,75,CUS_0x1069,2023-01-01
2,-13,8,87,166,214,-98,215,152,129,139,...,26,86,171,125,-130,354,17,302,CUS_0x114a,2023-01-01
3,-85,45,200,89,128,54,76,51,61,139,...,172,96,174,163,37,207,180,118,CUS_0x1184,2023-01-01
4,55,120,226,-86,253,97,107,68,103,126,...,76,43,183,159,-26,104,118,184,CUS_0x1297,2023-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8969,242,54,14,-84,86,-6,29,22,-6,52,...,197,58,71,61,-141,243,133,77,CUS_0xdf6,2023-01-01
8970,155,200,71,-79,221,223,74,-10,124,211,...,-12,155,10,24,178,49,156,-153,CUS_0xe23,2023-01-01
8971,143,2,42,248,163,-13,-13,20,183,183,...,202,64,105,289,51,77,-20,89,CUS_0xe4e,2023-01-01
8972,109,161,161,29,57,122,-61,223,66,0,...,101,125,249,116,138,-8,160,94,CUS_0xedd,2023-01-01


### Features Attributes

In [11]:
# create bronze datalake
bronze_attributes_directory = "datamart/bronze/attributes/"

if not os.path.exists(bronze_attributes_directory):
    os.makedirs(bronze_attributes_directory)

In [12]:
# No need to be partitioned
utils.attributes.attributes_data_processing_bronze_table.process_bronze_table(bronze_attributes_directory, spark).toPandas()

row count: 12500
saved to: datamart/bronze/attributes/bronze_attributes.csv


,Customer_ID,Name,Age,SSN,Occupation,snapshot_date
0,CUS_0x1000,Alistair Barrf,18,913-74-1218,Lawyer,2023-05-01
1,CUS_0x1009,Arunah,26,063-67-6938,Mechanic,2025-01-01
2,CUS_0x100b,Shirboni,19,#F%$D@*&8,Media_Manager,2024-03-01
3,CUS_0x1011,Schneyerh,44,793-05-8223,Doctor,2023-11-01
4,CUS_0x1013,Cameront,44,930-49-9615,Mechanic,2023-12-01
...,...,...,...,...,...,...
12495,CUS_0xff3,Somervilled,55,#F%$D@*&8,Scientist,2024-06-01
12496,CUS_0xff4,Poornimaf,37,655-05-7666,Entrepreneur,2024-12-01
12497,CUS_0xff6,Shieldsb,19,541-92-8371,Doctor,2024-10-01
12498,CUS_0xffc,Brads,18,226-86-7294,Musician,2024-01-01


### Features Financials

In [13]:
# create bronze datalake
bronze_financials_directory = "datamart/bronze/financials/"

if not os.path.exists(bronze_financials_directory):
    os.makedirs(bronze_financials_directory)

In [14]:
# No need to be partitioned
utils.financials.financials_data_processing_bronze_table.process_bronze_table(bronze_financials_directory, spark).toPandas()

row count: 12500
saved to: datamart/bronze/financials/bronze_financials.csv


,Customer_ID,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,snapshot_date
0,CUS_0x1000,30625.94,2706.161667,6,5,27,2,"Credit-Builder Loan, and Home Equity Loan",57,26,...,Bad,1562.91,30.077191,10 Years and 9 Months,Yes,42.941090,77.31427572208112,High_spent_Medium_value_payments,400.36080052211616,2023-05-01
1,CUS_0x1009,52312.68_,4250.390000,6,5,17,4,"Not Specified, Home Equity Loan, Credit-Builde...",5,18,...,_,202.68,40.286997,31 Years and 0 Months,Yes,108.366467,58.66019164829086,High_spent_Medium_value_payments,508.01234122645366,2025-01-01
2,CUS_0x100b,113781.38999999998,9549.782500,1,4,1,0,None,14,8,...,Good,1030.2,28.592943,15 Years and 10 Months,No,0.000000,617.0792665202719,High_spent_Small_value_payments,597.8989834797281,2024-03-01
3,CUS_0x1011,58918.47,5208.872500,3,3,17,3,"Student Loan, Credit-Builder Loan, and Debt Co...",27,13,...,Standard,473.14,27.829959,15 Years and 10 Months,Yes,123.434939,383.35084463651407,Low_spent_Medium_value_payments,294.1014665671429,2023-11-01
4,CUS_0x1013,98620.98,7962.415000,3,3,6,3,"Student Loan, Debt Consolidation Loan, and Per...",12,9,...,Good,1233.51,26.524864,17 Years and 10 Months,No,228.018084,332.3337079767732,High_spent_Medium_value_payments,485.8897083704929,2023-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12495,CUS_0xff3,17032.785,1176.398750,0,6,2,3,"Personal Loan, Mortgage Loan, and Auto Loan",13,7_,...,Good,1229.08,26.915577,17 Years and 3 Months,No,33.299764,81.19585741743609,Low_spent_Small_value_payments,293.14425404017936,2024-06-01
12496,CUS_0xff4,25546.26,2415.855000,8,7,14,5_,"Not Specified, Student Loan, Student Loan, Cre...",15,13_,...,Standard,758.44,39.333488,18 Years and 9 Months,Yes,101.328637,189.81586133347676,Low_spent_Medium_value_payments,230.44100130267208,2024-12-01
12497,CUS_0xff6,117639.92,9727.326667,5,6,1,2,"Home Equity Loan, and Auto Loan",-3,7,...,Good,338.3,32.871853,24 Years and 11 Months,No,126.638453,534.0885271982645,Low_spent_Medium_value_payments,592.0056861539196,2024-10-01
12498,CUS_0xffc,60877.17,5218.097500,6,8,27,8,"Credit-Builder Loan, Payday Loan, Not Specifie...",46,14,...,_,1300.13,29.026498,13 Years and 1 Months,Yes,272.809169,46.4256138380274,High_spent_Large_value_payments,442.5749666782518,2024-01-01


## Build Silver Table

### LMS

In [15]:
# create bronze datalake
silver_loan_daily_directory = "datamart/silver/loan_daily/"

if not os.path.exists(silver_loan_daily_directory):
    os.makedirs(silver_loan_daily_directory)

In [16]:
# run silver backfill
for date_str in dates_str_lst:
    utils.lms.lms_data_processing_silver_table.process_silver_table(date_str, bronze_lms_directory, silver_loan_daily_directory, spark)

loaded from: datamart/bronze/lms/bronze_loan_daily_2023_01_01.csv row count: 530


saved to: datamart/silver/loan_daily/silver_loan_daily_2023_01_01.parquet
loaded from: datamart/bronze/lms/bronze_loan_daily_2023_02_01.csv row count: 1031
saved to: datamart/silver/loan_daily/silver_loan_daily_2023_02_01.parquet
loaded from: datamart/bronze/lms/bronze_loan_daily_2023_03_01.csv row count: 1537
saved to: datamart/silver/loan_daily/silver_loan_daily_2023_03_01.parquet
loaded from: datamart/bronze/lms/bronze_loan_daily_2023_04_01.csv row count: 2047
saved to: datamart/silver/loan_daily/silver_loan_daily_2023_04_01.parquet
loaded from: datamart/bronze/lms/bronze_loan_daily_2023_05_01.csv row count: 2568
saved to: datamart/silver/loan_daily/silver_loan_daily_2023_05_01.parquet
loaded from: datamart/bronze/lms/bronze_loan_daily_2023_06_01.csv row count: 3085
saved to: datamart/silver/loan_daily/silver_loan_daily_2023_06_01.parquet
loaded from: datamart/bronze/lms/bronze_loan_daily_2023_07_01.csv row count: 3556
saved to: datamart/silver/loan_daily/silver_loan_daily_2023_07_0

saved to: datamart/silver/loan_daily/silver_loan_daily_2023_09_01.parquet
loaded from: datamart/bronze/lms/bronze_loan_daily_2023_10_01.csv row count: 4978


saved to: datamart/silver/loan_daily/silver_loan_daily_2023_10_01.parquet
loaded from: datamart/bronze/lms/bronze_loan_daily_2023_11_01.csv row count: 5469


saved to: datamart/silver/loan_daily/silver_loan_daily_2023_11_01.parquet
loaded from: datamart/bronze/lms/bronze_loan_daily_2023_12_01.csv row count: 5428


...

saved to: datamart/silver/loan_daily/silver_loan_daily_2024_07_01.parquet
loaded from: datamart/bronze/lms/bronze_loan_daily_2024_08_01.csv row count: 5531
saved to: datamart/silver/loan_daily/silver_loan_daily_2024_08_01.parquet
loaded from: datamart/bronze/lms/bronze_loan_daily_2024_09_01.csv row count: 5537
saved to: datamart/silver/loan_daily/silver_loan_daily_2024_09_01.parquet
loaded from: datamart/bronze/lms/bronze_loan_daily_2024_10_01.csv row count: 5502
saved to: datamart/silver/loan_daily/silver_loan_daily_2024_10_01.parquet
loaded from: datamart/bronze/lms/bronze_loan_daily_2024_11_01.csv row count: 5501
saved to: datamart/silver/loan_daily/silver_loan_daily_2024_11_01.parquet
loaded from: datamart/bronze/lms/bronze_loan_daily_2024_12_01.csv row count: 5531
saved to: datamart/silver/loan_daily/silver_loan_daily_2024_12_01.parquet
loaded from: datamart/bronze/lms/bronze_loan_daily_2025_01_01.csv row count: 5539


saved to: datamart/silver/loan_daily/silver_loan_daily_2025_01_01.parquet
loaded from: datamart/bronze/lms/bronze_loan_daily_2025_02_01.csv row count: 5028
saved to: datamart/silver/loan_daily/silver_loan_daily_2025_02_01.parquet
loaded from: datamart/bronze/lms/bronze_loan_daily_2025_03_01.csv row count: 4515
saved to: datamart/silver/loan_daily/silver_loan_daily_2025_03_01.parquet
loaded from: datamart/bronze/lms/bronze_loan_daily_2025_04_01.csv row count: 4024
saved to: datamart/silver/loan_daily/silver_loan_daily_2025_04_01.parquet
loaded from: datamart/bronze/lms/bronze_loan_daily_2025_05_01.csv row count: 3526


saved to: datamart/silver/loan_daily/silver_loan_daily_2025_05_01.parquet
loaded from: datamart/bronze/lms/bronze_loan_daily_2025_06_01.csv row count: 3021
saved to: datamart/silver/loan_daily/silver_loan_daily_2025_06_01.parquet
loaded from: datamart/bronze/lms/bronze_loan_daily_2025_07_01.csv row count: 2478
saved to: datamart/silver/loan_daily/silver_loan_daily_2025_07_01.parquet
loaded from: datamart/bronze/lms/bronze_loan_daily_2025_08_01.csv row count: 1985
saved to: datamart/silver/loan_daily/silver_loan_daily_2025_08_01.parquet
loaded from: datamart/bronze/lms/bronze_loan_daily_2025_09_01.csv row count: 1529


saved to: datamart/silver/loan_daily/silver_loan_daily_2025_09_01.parquet
loaded from: datamart/bronze/lms/bronze_loan_daily_2025_10_01.csv row count: 1041
saved to: datamart/silver/loan_daily/silver_loan_daily_2025_10_01.parquet
loaded from: datamart/bronze/lms/bronze_loan_daily_2025_11_01.csv row count: 526


saved to: datamart/silver/loan_daily/silver_loan_daily_2025_11_01.parquet


In [17]:
utils.lms.lms_data_processing_silver_table.process_silver_table(dates_str_lst[0], bronze_lms_directory, silver_loan_daily_directory, spark).toPandas()

loaded from: datamart/bronze/lms/bronze_loan_daily_2023_01_01.csv row count: 530


saved to: datamart/silver/loan_daily/silver_loan_daily_2023_01_01.parquet


,loan_id,Customer_ID,loan_start_date,tenure,installment_num,loan_amt,due_amt,paid_amt,overdue_amt,balance,snapshot_date,mob,installments_missed,first_missed_date,dpd
0,CUS_0x1037_2023_01_01,CUS_0x1037,2023-01-01,10,0,10000.0,0.0,0.0,0.0,10000.0,2023-01-01,0,0,None,0
1,CUS_0x1069_2023_01_01,CUS_0x1069,2023-01-01,10,0,10000.0,0.0,0.0,0.0,10000.0,2023-01-01,0,0,None,0
2,CUS_0x114a_2023_01_01,CUS_0x114a,2023-01-01,10,0,10000.0,0.0,0.0,0.0,10000.0,2023-01-01,0,0,None,0
3,CUS_0x1184_2023_01_01,CUS_0x1184,2023-01-01,10,0,10000.0,0.0,0.0,0.0,10000.0,2023-01-01,0,0,None,0
4,CUS_0x1297_2023_01_01,CUS_0x1297,2023-01-01,10,0,10000.0,0.0,0.0,0.0,10000.0,2023-01-01,0,0,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525,CUS_0xe98_2023_01_01,CUS_0xe98,2023-01-01,10,0,10000.0,0.0,0.0,0.0,10000.0,2023-01-01,0,0,None,0
526,CUS_0xea6_2023_01_01,CUS_0xea6,2023-01-01,10,0,10000.0,0.0,0.0,0.0,10000.0,2023-01-01,0,0,None,0
527,CUS_0xed3_2023_01_01,CUS_0xed3,2023-01-01,10,0,10000.0,0.0,0.0,0.0,10000.0,2023-01-01,0,0,None,0
528,CUS_0xed8_2023_01_01,CUS_0xed8,2023-01-01,10,0,10000.0,0.0,0.0,0.0,10000.0,2023-01-01,0,0,None,0
